Stage 3: Python  notebook: Demonstrate simple queries in Python notebooks over your data. In particular, apply sentiment analysis to data in different dates and periods and generate simple plots. Enable search queries over the data.